# Deploy de Modelos de Machine Learning

### Exemplo de deployment utilizando Azure Databricks e Azure Machine Learning

Neste tutorial demonstraremos como realizar o deployment de modelos de Machine Learning com modelos já treinados anteriormente. 

Utilizaremos o [Azure Databricks](https://azure.microsoft.com/pt-br/services/databricks/) para carregamento dos modelos e posterior criação das APIs de consumo realizando sua conexão com o [Azure Machine Learning](https://azure.microsoft.com/en-us/services/machine-learning/). 

Este tutorial foi inspirado nos notebooks [MLflow Quick Start Part 1: Training and Logging](https://docs.azuredatabricks.net/_static/notebooks/mlflow/mlflow-quick-start-training.html) e [Quick Start Part 2: Serving Models with Azure ML](https://docs.azuredatabricks.net/_static/notebooks/mlflow/mlflow-quick-start-deployment-azure.html). Para maiores detalhes aconselho fortemente sua leitura 🤓

### Modelo de Machine Learning de Regressão Linear para previsão do preço de venda de Carros Usados
Utilizaremos neste tutorial um modelo de Regressão Linear Simples já previamente treinado com dados de vendas de veículos usados. Os passos para treinamento e persistência deste modelo podem ser encontrados [aqui](https://github.com/lfbraz/machine-learning-tutorial/blob/master/notebooks/ml-modelo-regressao-carros-usados.ipynb).

Este tutorial pode ser utilizado com outros tipos de modelo de Machine Learning para atuação em diferentes cenários (Problemas de classificação, NLP, etc) em que os passos serão semalhantes (basta previamente ter persistido o modelo).

Fique a vontade para adaptar conforme sua necessidade 😉

## Instalação das bibliotecas
Será necessário o uso da biblioteca OpenSource [`mlflow`](https://mlflow.org/) que permite a organização e monitoramento de métricas e execuções associadas ao treinamento dos modelos de Machine Learning, e também o [`azureml-sdk`](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py) que permite a criação e manipulação de Workspaces do [Azure Machine Learning](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py) utilizando linguagem Python.

Para instalação das bibliotecas no Databricks utilizaremos o comando `dbutils.library.installPyPi` que basicamente instala os pacotes utilizando o repositório [`PyPi`](https://pypi.org/). No Databricks esta forma de instalação de bibliotecas faz com que elas sejam instaladas apenas no contexto do próprio notebook. Para outros métodos de instalação (Cluster, Workspace, etc) consulte a [documentação](https://docs.microsoft.com/en-us/azure/databricks/libraries#library-modes).

Caso esteja utilizando outras plataformas para desenvolvimento de modelos, utilizar o comando apropriado para instalação das bibliotecas seguindo o repositório `PyPi`.

In [4]:
dbutils.library.installPyPI("mlflow", extras="extras")

dbutils.library.installPyPI("azureml-sdk")
dbutils.library.restartPython()

##Carregamento do modelo de Machine Learning previamente treinado
Conforme dito anteriormente utilizaremos um modelo [já treinado](https://github.com/lfbraz/machine-learning-tutorial/blob/master/notebooks/ml-modelo-regressao-carros-usados.ipynb) em que basicamente adaptaremos o caminho do diretório (do modelo persistido) nas variáveis `NOME_MODELO` e `CAMINHO_MODELO`.

Com isso, faremos o carregamento do modelo utilizando o `LinearRegressionModel`. <br/><br/>

* **IMPORTANTE**: É necessário que você saiba previamente qual o tipo de modelo que será carregado para que possa instanciar da forma correta a biblioteca mais apropriada. No caso deste tutorial sabíamos previamente se tratar de um modelo de Regressão Linear treinado utilizando PySpark.

In [6]:
from pyspark.ml import PipelineModel

NOME_MODELO = 'modelo_regressao_linear.model'
CAMINHO_MODELO = '/mnt/models/{}'.format(NOME_MODELO)

model = PipelineModel.load(CAMINHO_MODELO)

###Carregar os dados para *"PREVISÃO"* do modelo
Agora vamos carregar alguns dados para que seja possível *testarmos* o modelo que carregamos. Como o objetivo deste tutorial é apenas analisarmos o processo de *deployment* utilizaremos o mesmo [arquivo](https://github.com/lfbraz/machine-learning-tutorial/blob/master/datasets/dataset-carros-usados.csv) de treino do modelo que já havia sido previamente importado para o Databricks (vide seção *Importar base de dados* do [link](https://github.com/lfbraz/machine-learning-tutorial/blob/master/notebooks/ml-modelo-regressao-carros-usados.ipynb)).

In [8]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"
output_dbfs_file_path = '/data/dataset-carros-usados.csv'

carros_usados = spark.read.format(file_type) \
                     .option("inferSchema", infer_schema) \
                     .option("header", first_row_is_header) \
                     .option("sep", delimiter) \
                     .load(output_dbfs_file_path)

### Tratamento dos dados
Assim como na etapa de *treinamento* do modelo, precisamos agora realizar o *tratamento dos dados* antes da predição.

Os tratamentos que foram realizados em um `pipeline` não precisam ser refeitos (pois serão aplicados diretamente pelo modelo). 
Aqui apenas precisamos remover os valores *NULOS* (conforme realizado no [tutorial](https://github.com/lfbraz/machine-learning-tutorial/blob/master/notebooks/ml-modelo-regressao-carros-usados.ipynb)).

Os dados tratados serão segmentados randomicamente em duas bases chamadas `validacao1` e `validacao2` utilizando o mesmo arquivo de dados utilizados no treinamento (faremos isso apenas para facilitar a demonstração, pois o objetivo aqui não é avaliar a performance do modelo).

In [10]:
carros_usados = carros_usados.na.drop()
validacao1, validacao2 = carros_usados.randomSplit([0.5, 0.5])

In [11]:
predicoes = model.transform(validacao1)
predicoes.show(n=5)

### Predição
Agora podemos chamar o método `transform` do modelo carregado para realizar as predições `ad-hoc`.

In [13]:
predicoes = model.transform(validacao1)
predicoes.show(n=5)

##Persistir o modelo utilizando o MLFlow
Agora precisamos utilizar a biblioteca `mlflow` para persistirmos o modelo e registrarmos uma execução associada a ele. Esta etapa é importante pois utilizaremos posteriormente o `run_id` vinculado a esta execução. Precisamos também associar um nome de modelo (variável `NOME_MODELO_DEPLOY`) e caminho (variável `modelpath`) onde este será persistido (agora no formato mlflow).

Neste tutorial estamos utilizando um modelo `spark` porém existem outras possibilidades por exemplo utilizando `sklearn`, entre outros.

In [15]:
import mlflow.spark
NOME_MODELO_DEPLOY = 'modelo-regressao-linear-tutorial'

with mlflow.start_run():
  # Log the model
  mlflow.spark.log_model(model, NOME_MODELO_DEPLOY)
  
  # Persist the model in dbfs
  modelpath = '/dbfs/mlflow/model-linear-regression/{}'.format(NOME_MODELO_DEPLOY)
  mlflow.spark.save_model(model, modelpath)

## Criar/Utilizar Workspace do Azure Machine Learning
Utilizamos o Azure Machine Learning para disponibilização dos `endpoints` das APIs que irão consumir os modelos de Machine Learning. Para interação com ele, vamos utilizar o [Azure Machine Learning SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py) em Python, em que é possível criar novas Workspaces (ou utilizar Workspaces existentes) para facilitar o processo de deployment.

É necessário preencher as variáveis `WORKSPACE_NAME`, `WORKSPACE_LOCATION`, `RESOURCE_GROUP` e `SUBSCRIPTION_ID` com os dados da assinatura Azure que será utilizada para disponibilização do Azure Machine Learning.

Para o `run_id` deve-se pegar coletar este dado a partir da execução realizada acima:

Para isso seguir o exemplo abaixo:

1. Clicar no ícone **Runs** na barra de contexto do Notebook. Nele é possível visualizar as execuções, métricas e parâmetros. Por exemplo: <img src="https://docs.databricks.com/_static/images/mlflow/mlflow-notebook-experiments.gif"/>
   
1. Clicar no ícone **External Link** <img src="https://docs.databricks.com/_static/images/external-link.png"/> in the Runs context bar to view the notebook experiment. For example: <img src="https://docs.databricks.com/_static/images/mlflow/quick-start-nb-experiment.png"/>

![image](https://docs.azuredatabricks.net/_static/images/mlflow/mlflow-deployment-example-run-info.png)

Com isso podemos agora construir a imagem contendo o modelo e disponibilizá-lo no Azure Machine Learning.

In [18]:
import azureml
from azureml.core import Workspace
import mlflow.azureml

WORKSPACE_NAME = '<NOME_WORKSPACE>'
WORKSPACE_LOCATION = '<REGIAO_WORKSPACE>'
RESOURCE_GROUP = '<NOME_RESOURCE_GROUP>'
SUBSCRIPTION_ID = '<ID_SUBSCRICAO>'

NOME_IMAGEM = 'model-image-linear-regression'
DESCRICAO_IMAGEM = 'Modelo de Regressão Linear - Tutorial'

workspace = Workspace.create(name = WORKSPACE_NAME,
                             location = WORKSPACE_LOCATION,
                             resource_group = RESOURCE_GROUP,
                             subscription_id = SUBSCRIPTION_ID,
                             exist_ok=True)

run_id1 = '<RUN_ID>'
model_uri = 'runs:/' + run_id1 + '/{}'.format(NOME_MODELO_DEPLOY)

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name=NOME_MODELO_DEPLOY,
                                                      image_name=NOME_IMAGEM,
                                                      description=DESCRICAO_IMAGEM,
                                                      synchronous=False)

model_image.wait_for_creation(show_output=True)

Por padrão será solicitada a autenticação utilizando o `Interactive Login`. Para cenários produtivos deve-se utilizar um registro de aplicação com `Service Principal`. Na [documentação](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication#set-up-service-principal-authentication) temos uma explicação mais detalhada dos diferentes tipos de autenticação.

#Deploy
Agora com a imagem criada, podemos escolher dois tipos de deployment, utilizando `ACI` (Azure Container Image) ou `AKS` (Azure Kubernetes Service).

Para cenários de desenvolvimento é indicado o uso do `ACI` já para cenários produtivos `AKS` terá melhores opções quanto a segurança e escalabilidade.

##ACI - Azure Container Image
Abaixo será demonstrado como criar um `endpoint` utilizando o `ACI`. Lembrando que estaremos utilizando o Workspace instanciado no passo anterior.

In [22]:
from azureml.core.webservice import AciWebservice, Webservice

NOME_WEBSERVICE = 'linear-model-webservice'

#TODO: Change deploy_from_image to environments (https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)

dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=NOME_WEBSERVICE, 
                                              image=model_image, 
                                              deployment_config=dev_webservice_deployment_config, 
                                              workspace=workspace)

dev_webservice.wait_for_deployment()

Com isso a API já deverá ter sido criada a partir de um `ACI`. Podemos analisar no Azure Machine Learning o modelo criado:

<img src="https://github.com/lfbraz/machine-learning-tutorial/blob/master/images/azure-ml-models.PNG?raw=true"/>

E também o endpoint:

<img src="https://github.com/lfbraz/machine-learning-tutorial/blob/master/images/azure-ml-endpoints.PNG?raw=true"/>

###Predição com o uso da API (Real-time endpoint)
Agora vamos utilizar o DataFrame `validacao1` (removendo a coluna target *PRECO*) para validação da API criada no Azure Machine Learning. 

A API espera que o *INPUT* esteja no formato `json`, então vamos realizar a formatação convertendo para um DataFrame `pandas` que possui o método `to_json`, facilitando a execução deste processo. Também é necessário que seja utilizado o parâmetro `orient="split"` para que a formatação do `json` esteja de acordo com o que é esperado pela API.

Neste exemplo, faremos uma chamada utilizando apenas uma linha do DataFrame original (`validacao1`), porém a API suporta a execução em `batch` com várias linhas sendo enviadas no mesmo `json` (para isso basta aumentar o range de seleção no método `iloc`).

In [25]:
sample = validacao1.toPandas().drop('PRECO', axis=1)
sample = sample.iloc[0:1,:]
query_input = sample.to_json(orient="split")

# Chamada da API
Finalmente faremos o *request* da API utilizando a variável `query_input`. A URL da API pode ser obtida através do `dev_webservice.scoring_uri` que foi gerado no deploy do endpoint.

In [27]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
  
  print('URI: {}'.format(scoring_uri))
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.loads(json.dumps(inputs)), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Também é possível utilizar a API via qualquer aplicativo cliente para chamadas HTTP (curl, postman, etc).

## Azure Kubernetes Services (AKS)
Para cenários produtivos, uma melho opção de deploy é utilizando um [`AKS`](https://azure.microsoft.com/pt-br/overview/kubernetes-getting-started/) que traz maiores benefícios quanto a segurança e escalabilidade.

Neste cenário com `AKS` é possível seguirmos com o Deploy de duas formas: Criando um novo cluster AKS ou realizando o deploy em um cluster existente. Neste tutorial demonstraremos a primeira opção. No [link](https://docs.azuredatabricks.net/_static/notebooks/mlflow/mlflow-quick-start-deployment-azure.html) é possível analisar o processo utilizando um cluster já existente (seção *"Connect to an existing AKS cluster"*).

####Criar um novo Cluster

In [31]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "aks-cluster"

# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Agora podemos fazer o deploy do webservice utilizando o novo cluster AKS

In [33]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = "linear-regression-model-prod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

E agora realizar uma chamada da API para testarmos seu funcionamento. Neste caso será necessário utilizar um `key` para autenticação da API que pode ser obtida através do método `get_keys()`.

In [35]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [36]:
query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)